
<a href="https://colab.research.google.com/github/edumunozsala/llamaindex-RAG-techniques/blob/main/chain-of-abstraction-llamapack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chain-of-Abstraction LlamaPack for RAG 

Last February, to enhance Large Language Models (LLMs) with reasoning abilities, researchers from EPFL, FAIR and Meta have proposed a novel approach known as Chain-of-Abstraction (CoA). This method addresses the challenge of enabling LLMs to effectively utilize external knowledge sources in an efficient multistep reasoning process, by decoupling reasoning and the retrieval of granular knowledge.

While tools aid LLMs in accessing external knowledge, fine-tuning LLM agents to adeptly invoke tools in multi-step reasoning poses challenges. Interconnected tool calls necessitate comprehensive and efficient tool usage planning. CoA presents a novel strategy for LLMs to optimize tool utilization in multi-step reasoning scenarios. The method involves training LLMs to initially decode reasoning chains using abstract placeholders. Subsequently, domain-specific tools are employed to concretize each reasoning chain by filling in specific knowledge.

Planning with abstract chains enables LLMs to learn more generalized reasoning strategies. These strategies exhibit robustness to shifts in domain knowledge, such as mathematical results relevant to diverse reasoning questions. Furthermore, CoA facilitates parallel decoding and tool calling, mitigating inference delays associated with awaiting tool responses.

The chain-of-abstraction (CoA) LlamaPack implements a generalized version of the strategy. By prompting the LLM to write function calls in a chain-of-thought format, the agent can execute both simple and complex combinations of function calls needed to execute a task.

From there, the function calls can be parsed into a dependency graph, and executed. Then, the values in the CoA are replaced with their actual results. As an extension to the original paper, the agent also run the LLM a final time, to rewrite the response in a more readable and user-friendly way.

Original paper:[Efficient Tool Use with Chain-of-Abstraction Reasoning](https://arxiv.org/abs/2401.17464)

Original code from Llama-index: https://docs.llamaindex.ai/en/stable/examples/agent/coa_agent/


## Setup

The libraries to install

In [ ]:
!pip install llama-index-core llama-index-llms-openai llama-index-embeddings-openai
!pip install llama-index-packs-agents-coa

In [1]:
import os

# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()


Load the API Keys:

In [2]:
from dotenv import load_dotenv

# Load the enviroment variables
load_dotenv()

True

### Import the Libraries

In [3]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool
from llama_index.packs.agents_coa import CoAAgentPack

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter

## Load the data and create the indexes

For this demo, we are going to extract information from a document about census in Spain in 2023 and the other document describes the census and population figures for the EU in 2023. Then, we can build a query engine to extract information for Spain and we can also create an engine to get data for other countries in the EU to compare figures.

We read both documents, extract chunks of text and build and index to retrieve information.

In [5]:
text_splitter = SentenceSplitter(chunk_size=256, chunk_overlap=20)

if not os.path.exists("./storage/cspain"):
    # load data
    spain_docs = SimpleDirectoryReader(
        input_files=["./data/census/census_spain_2023_en.pdf"]
    ).load_data()
    
    # build index
    spain_index = VectorStoreIndex.from_documents(spain_docs, transformations=[text_splitter])
    # persist index
    spain_index.storage_context.persist(persist_dir="./storage/cspain")
else:
        spain_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir="./storage/cspain"),
        )
        


if not os.path.exists("./storage/ceu"):
    eu_docs = SimpleDirectoryReader(
        input_files=["./data/census/census_UE_2023.pdf"]
    ).load_data()

    # build index
    eu_index = VectorStoreIndex.from_documents(eu_docs,transformations=[text_splitter])
    # persist index
    eu_index.storage_context.persist(persist_dir="./storage/ceu")
else:
        eu_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir="./storage/ceu"),
        )


In [8]:
spain_index.ref_doc_info

{'3e9de16e-163a-4400-a1b0-904831dac5fa': RefDocInfo(node_ids=['a470db86-0f97-4362-9e60-a2cb3ad758ca', '1b6e8386-298d-4f40-a399-9261bd75c2a6', '6955027d-2fa9-496a-9f44-b2b665318eee'], metadata={'page_label': '1', 'file_name': 'census_spain_2023_en.pdf', 'file_path': 'data\\census\\census_spain_2023_en.pdf', 'file_type': 'application/pdf', 'file_size': 196223, 'creation_date': '2024-04-18', 'last_modified_date': '2024-04-18'}),
 'f3c42450-d353-4463-ad43-d952fd5bac25': RefDocInfo(node_ids=['36a58d9f-1122-4fe3-8740-cbae34eeff7f', '66fdcb70-79f0-4d02-819b-7102bf65d4b8', '6fe12936-18e6-4303-81a4-4b63f6c556a8', '083ebc09-603b-418b-a1e6-8c617fff0394', '8cdfffc1-c93d-489e-a77c-093fe9fe7e97', '3a0d9aae-2fb8-4974-9c4e-66aed1e6e208'], metadata={'page_label': '2', 'file_name': 'census_spain_2023_en.pdf', 'file_path': 'data\\census\\census_spain_2023_en.pdf', 'file_type': 'application/pdf', 'file_size': 196223, 'creation_date': '2024-04-18', 'last_modified_date': '2024-04-18'}),
 'e36e5eb0-4498-4ee2

## Create the Query Tools for the Agent

Then, we create two query engines and tools, one to answer questions about Spain and the other one about the EU. 

In [15]:
spain_engine = spain_index.as_query_engine(similarity_top_k=2)
eu_engine = eu_index.as_query_engine(similarity_top_k=2)

query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=spain_engine,
        name="spain_census",
        description=(
            "Provides demographic information and census about Spain for year 2022. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=eu_engine,
        name="ue_census",
        description=(
            "Provides demographic information and census about countries in the EU for year 2022. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
]

## Run the Chain of Abstraction Agent from LlamaPack

Thanks to LlamaPack, we can build a Chain of Abstraction Agent in one line of code. The Agent will describe the steps taken to get the final answer.

In [10]:
# Set the embedding model
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small", embed_batch_size=256
)
# Set the LLM
Settings.llm = OpenAI(model="gpt-4-turbo", temperature=0.1)
# Create ther CoA Agent
pack = CoAAgentPack(tools=query_engine_tools, llm=Settings.llm)

In [13]:
#response = pack.run("How did Spains foreign people growth compare to Frances in 2022?")
response = pack.run("Was Spains increase in population greater than in the EU?")

==== Available Parsed Functions ====
def spain_census(input: string):
   """Provides demographic information and census about Spain for year 2022. Use a detailed plain text question as input to the tool."""
    ...
def ue_census(input: string):
   """Provides demographic information and census about countries in the EU for year 2022. Use a detailed plain text question as input to the tool."""
    ...
==== Generated Chain of Abstraction ====
Abstract plan of reasoning:
1. Obtain the population increase of Spain in 2022 by querying the Spain census function with a specific question about population growth: [FUNC spain_census("What was the population increase in Spain in 2022?") = y1].
2. Obtain the population increase of the European Union (EU) in 2022 by querying the EU census function with a specific question about population growth across the EU: [FUNC ue_census("What was the population increase in the EU in 2022?") = y2].
3. Compare the results from y1 and y2 to determine if Spain's 

In [14]:
print(str(response))

Based on the data obtained, Spain's population increase in 2022 was almost 600,000 people. When comparing this to the population increase in the European Union, which was reported as 1.7 per 1,000 persons across 27 countries, we need to consider the total population of the EU to make a direct comparison. Without the total EU population figure, it's challenging to calculate the exact number increase for the EU. However, given the substantial figure reported for Spain, it suggests that Spain's increase was notably significant, potentially surpassing the average increase rate in the EU, especially if the EU's total increase translates into a smaller absolute number when distributed across its much larger population base. Thus, it is likely that Spain's population increase was greater than that of the EU in 2022.
